In [5]:
# Import des packages nécessaires
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import scipy.integrate as integrate
from scipy.stats import uniform, norm
import scipy.stats as sps
from random import choices

In [6]:
# Définition des fonctions

def changementeps(distances, eps, poids, alpha, n):
    epsilons = np.sort(distances[distances < eps])[::-1]
    nbops = len(epsilons)
    ESS = 1 / sum(poids ** 2)
    k = 0
    nouvpoids = poids
    while k < nbops - 1:
        nouveps = epsilons[k]
        for i in range(n):
            if poids[i] != 0:
                if sum(distances[:, i][np.where(distances[:, i] <= eps)]) == 0:
                    nouvpoids[i] = 0
                else:
                    nouvpoids[i] = poids[i] * sum(distances[:, i][np.where(distances[:, i] <= nouveps)]) / sum(
                        distances[:, i][np.where(distances[:, i] <= eps)])
        nouvpoids = nouvpoids / sum(nouvpoids)
        nouvESS = 1 / sum(nouvpoids ** 2)
        if nouvESS < alpha * ESS:
            return nouveps, nouvpoids, nouvESS
        k += 1
    return eps, poids, ESS

def chgtpar(simui, distancesi, sd, eps, data, dprior, dist, n, means, M):
    simu = simui + np.random.multivariate_normal(np.zeros(len(sd)), cov=sd)
    ndistances = np.tile([dist(simu)], M)
    u = np.random.uniform(1)
    if u < (
            sum(ndistances[np.where(ndistances <= eps)] / sum(
                distancesi[np.where(distancesi <= eps)]) * dprior(simu, means) / dprior(simui, means))):
        return simu, ndistances
    else:
        return simui, distancesi

def pasnoyau(simu, eps, distances, poids, data, dprior, dist, n, means, M):
    sd = 2 * np.cov(np.transpose(simu))
    mdistances = np.copy(distances)
    for i in range(n):
        if poids[i] > 0:
            simu[i], mdistances[:i] = chgtpar(simu[i], mdistances[:i], sd, eps, data, dprior, dist, n, means, M)
    return simu, mdistances

def smc(data, n, M, T, objectifeps, alpha, rprior, dprior, dist, l):
    simu, means = rprior(n)
    distances = np.tile([dist(i) for i in simu], (M, 1))
    j = 1
    poids = np.array([1 / n] * n)
    ESS = n
    eps = np.max(distances)
    histeps = [eps]
    while j < T and eps > objectifeps:
        eps, poids, ESS = changementeps(distances, eps, poids, alpha, n)
        histeps.append(eps)
        if ESS < n / 2:
            simu = choices(simu, weights=poids, k=n)
            poids = np.array([1 / n] * n)
            ESS = n
            distances = np.tile([dist(i) for i in simu], (M, 1))
        simu, distances = pasnoyau(simu, eps, distances, poids, data, dprior, dist, n, means, M)
        j += 1
    simu = choices(simu, weights=poids, k=n)
    ESS = n
    distances = np.array([dist(i) for i in simu] * M)
    return simu, poids, distances, histeps

In [8]:
# Données et fonctions de statistiques

l = 50
data = np.random.normal(-5, 1, (20, l))

def rprior(n):
    means = np.random.uniform(-10, 10, size=n)
    res = [0] * n
    for i in range(n):
        res[i] = np.random.normal(means[i], 1, l)
    return res, means

statdata = np.array([np.percentile(data[i], np.arange(10, 91, 10)) for i in range(20)])

def dprior(simu, means):
    return np.sum(norm.pdf(simu, loc=means, scale=1))

def dist(simu):
    lev = np.arange(0.1, 0.91, 0.1)
    quantiles = np.array([[sps.norm(simu[i], 1).ppf(lev[j]) for j in range(9)] for i in range(20)])
    return np.sum(abs(quantiles - statdata))

# Exécution de l'algorithme
resultat_smc = smc(data=data, n=50, M=50, T=50, objectifeps=150, alpha=0.9, rprior=rprior, distprior=dprior, dist=dist, l)
print(resultat_smc)


C:\Users\sarah\AppData\Local\Temp\ipykernel_35568\2919879138.py:30: RuntimeWarning: divide by zero encountered in divide
  sum(ndistances[np.where(ndistances <= eps)] / sum(


([array([-4.79797423, -6.42069599, -3.32783823, -6.45950623, -3.53667633,
       -5.54238663, -3.26901929, -4.46034208, -3.16046177, -5.61507413,
       -4.96302543, -5.6646214 , -4.29834375, -5.29312106, -2.64426613,
       -4.44406898, -5.53005091, -7.15013837, -4.10522618, -4.65969096,
       -8.4615971 , -4.01017123, -4.49642972, -4.83212211, -2.76384451,
       -5.79861752, -5.13367249, -4.84309765, -6.14409577, -5.47986004,
       -4.8112253 , -3.98266794, -4.95708381, -3.42642774, -4.13295698,
       -4.78504154, -6.34193155, -3.48495313, -3.13229614, -4.64746833,
       -6.03435747, -4.64448269, -5.31322305, -4.51568278, -3.69765658,
       -3.4845128 , -3.42609198, -6.05935531, -5.31087149, -4.78582297]), array([-4.79797423, -6.42069599, -3.32783823, -6.45950623, -3.53667633,
       -5.54238663, -3.26901929, -4.46034208, -3.16046177, -5.61507413,
       -4.96302543, -5.6646214 , -4.29834375, -5.29312106, -2.64426613,
       -4.44406898, -5.53005091, -7.15013837, -4.10522618, -

In [ ]:
# Données et fonctions de statistiques

l = 50
data = np.random.normal(-5, 1, (20, l))

def rprior(n):
    means = np.random.uniform(-10, 10, size=n)
    res = [0] * n
    for i in range(n):
        res[i] = np.random.normal(means[i], 1, l)
    return res, means

statdata = np.array([np.percentile(data[i], np.arange(10, 91, 10)) for i in range(20)])

def dprior(simu, means):
    return np.sum(norm.pdf(simu, loc=means, scale=1))

def dist(simu):
    lev = np.arange(0.1, 0.91, 0.1)
    quantiles = np.array([[sps.norm(simu[i], 1).ppf(lev[j]) for j in range(9)] for i in range(20)])
    return np.sum(abs(quantiles - statdata))

In [ ]:
# Données et fonctions de statistiques

l = 50
data = np.random.normal(-5, 1, (20, l))

def rprior(n):
    means = np.random.uniform(-10, 10, size=n)
    res = [0] * n
    for i in range(n):
        res[i] = np.random.normal(means[i], 1, l)
    return res, means

statdata = np.array([np.percentile(data[i], np.arange(10, 91, 10)) for i in range(20)])

def dprior(simu, means):
    return np.sum(norm.pdf(simu, loc=means, scale=1))

def dist(simu):
    lev = np.arange(0.1, 0.91, 0.1)
    quantiles = np.array([[sps.norm(simu[i], 1).ppf(lev[j]) for j in range(9)] for i in range(20)])
    return np.sum(abs(quantiles - statdata))

In [ ]:
# Données et fonctions de statistiques

l = 50
data = np.random.normal(-5, 1, (20, l))

def rprior(n):
    means = np.random.uniform(-10, 10, size=n)
    res = [0] * n
    for i in range(n):
        res[i] = np.random.normal(means[i], 1, l)
    return res, means

statdata = np.array([np.percentile(data[i], np.arange(10, 91, 10)) for i in range(20)])

def dprior(simu, means):
    return np.sum(norm.pdf(simu, loc=means, scale=1))

def dist(simu):
    lev = np.arange(0.1, 0.91, 0.1)
    quantiles = np.array([[sps.norm(simu[i], 1).ppf(lev[j]) for j in range(9)] for i in range(20)])
    return np.sum(abs(quantiles - statdata))

In [ ]:
# Données et fonctions de statistiques

l = 50
data = np.random.normal(-5, 1, (20, l))

def rprior(n):
    means = np.random.uniform(-10, 10, size=n)
    res = [0] * n
    for i in range(n):
        res[i] = np.random.normal(means[i], 1, l)
    return res, means

statdata = np.array([np.percentile(data[i], np.arange(10, 91, 10)) for i in range(20)])

def dprior(simu, means):
    return np.sum(norm.pdf(simu, loc=means, scale=1))

def dist(simu):
    lev = np.arange(0.1, 0.91, 0.1)
    quantiles = np.array([[sps.norm(simu[i], 1).ppf(lev[j]) for j in range(9)] for i in range(20)])
    return np.sum(abs(quantiles - statdata))

In [ ]:
# Données et fonctions de statistiques

l = 50
data = np.random.normal(-5, 1, (20, l))

def rprior(n):
    means = np.random.uniform(-10, 10, size=n)
    res = [0] * n
    for i in range(n):
        res[i] = np.random.normal(means[i], 1, l)
    return res, means

statdata = np.array([np.percentile(data[i], np.arange(10, 91, 10)) for i in range(20)])

def dprior(simu, means):
    return np.sum(norm.pdf(simu, loc=means, scale=1))

def dist(simu):
    lev = np.arange(0.1, 0.91, 0.1)
    quantiles = np.array([[sps.norm(simu[i], 1).ppf(lev[j]) for j in range(9)] for i in range(20)])
    return np.sum(abs(quantiles - statdata))

In [ ]:
# Données et fonctions de statistiques

l = 50
data = np.random.normal(-5, 1, (20, l))

def rprior(n):
    means = np.random.uniform(-10, 10, size=n)
    res = [0] * n
    for i in range(n):
        res[i] = np.random.normal(means[i], 1, l)
    return res, means

statdata = np.array([np.percentile(data[i], np.arange(10, 91, 10)) for i in range(20)])

def dprior(simu, means):
    return np.sum(norm.pdf(simu, loc=means, scale=1))

def dist(simu):
    lev = np.arange(0.1, 0.91, 0.1)
    quantiles = np.array([[sps.norm(simu[i], 1).ppf(lev[j]) for j in range(9)] for i in range(20)])
    return np.sum(abs(quantiles - statdata))

In [ ]:

# ETAPE 10
np.random.normal(0, 1, (20, l))

# ETAPE 11
lev = np.arange(0.1, 0.91, 0.1)
np.array([[sps.norm(0, 1).ppf(lev[j]) for j in range(9)] for i in range(20)])

# ETAPE 12
data = np.random.normal(0, 1, (20, l))

def rprior(n):
    means = np.random.uniform(-10, 10, size=n)
    res = [0] * n
    for i in range(n):
        res[i] = np.random.normal(means[i], 1, l)
    return res, means

np.array([np.percentile(data[i], np.arange(10, 91, 10)) for i in range(20)])

# ETAPE 13
l = 50
data = np.random.normal(0, 1, (20, l))

def rprior(n):
    means = np.random.uniform(-10, 10, size=n)
    res = [0] * n
    for i in range(n):
        res[i] = np.random.normal(means[i], 1, l)
    return res, means

statdata = np.array([np.percentile(data[i], np.arange(10, 91, 10)) for i in range(20)])

def dprior(simu):
    return np.sum(norm.pdf(simu, loc=simu[1], scale=1))

def dist(simu):
    lev = np.arange(0.1, 0.91, 0.1)
    quantiles = np.array([[sps.norm(simu[i], 1).ppf(lev[j]) for j in range(9)] for i in range(20)])
    return np.sum(abs(quantiles - statdata))

dist(np.random.normal(0, 1, (l)))

# ETAPE 14
np.tile([1, 2, 3], (3, 1))[:, 2]
